In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import numpy as np

from pydrake.common import FindResourceOrThrow
from pydrake.geometry import Box, Role, SceneGraph
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import CoulombFriction, AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import RevoluteJoint, SpatialInertia, UnitInertia
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from pydrake.systems.primitives import TrajectorySource
from pydrake.systems.rendering import MultibodyPositionToGeometryPose
from pydrake.trajectories import PiecewisePolynomial

from meshcat import Visualizer

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)

# Sporadically need to run `pkill -f meshcat`

# Visualize Model

In [ ]:
model_file = "drake/planning/models/planar_iiwa_dense_collision_welded_gripper.yaml"
# model_file = "drake/planning/models/planar_iiwa_simple_collision_welded_gripper.yaml"

viz_role = Role.kIllustration
# viz_role = Role.kProximity

In [ ]:
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

table_dim = np.array([4, 4, 0.2])
table = plant.AddRigidBody("table", SpatialInertia(
        mass=1.0, p_PScm_E=np.array([0., 0., 0.]), G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
plant.WeldFrames(plant.world_frame(), table.body_frame(), RigidTransform(p=np.array([0, 0, -table_dim[2]/2])))
plant.RegisterVisualGeometry(table, RigidTransform(), Box(*table_dim), "table_vis",
                             np.array([0.5, 0.5, 0.5, 1.]))
plant.RegisterCollisionGeometry(table, RigidTransform(), Box(*table_dim), "table_collision",
                                CoulombFriction(0.9, 0.8))

directives_file = FindResourceOrThrow(model_file)
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa, wsg, shelves] = models

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)
diagram = builder.Build()

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(iiwa.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
sg_context = scene_graph.GetMyContextFromRoot(context)
diagram.Publish(context)

# Visualize Trajectory

In [ ]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()
    
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    table_dim = np.array([4, 4, 0.2])
    table = plant.AddRigidBody("table", SpatialInertia(
            mass=1.0, p_PScm_E=np.array([0., 0., 0.]), G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
    plant.WeldFrames(plant.world_frame(), table.body_frame(), RigidTransform(p=np.array([0, 0, -table_dim[2]/2])))
    plant.RegisterVisualGeometry(table, RigidTransform(), Box(*table_dim), "table_vis",
                                 np.array([0.5, 0.5, 0.5, 1.]))
    plant.RegisterCollisionGeometry(table, RigidTransform(), Box(*table_dim), "table_collision",
                                    CoulombFriction(0.9, 0.8))
    
    directives_file = FindResourceOrThrow(model_file)
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    
    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                       delete_prefix_on_load=False, role=viz_role)

    vis_diagram = builder.Build()
    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    
traj = PiecewisePolynomial.FirstOrderHold([0, 3], np.array([[-0.2, -1.2, 1.6], [0.4, 1.8, -0.5]]).T)

vis.delete()
display(vis.jupyter_cell())
visualize_trajectory(traj)